In [ ]:
import requests
import pandas as pb
import random
import collections
import string
import nltk
from nltk.util import ngrams
from scipy.sparse import csr_matrix
nltk.download('punkt')

# Assignment #2

# In This assignment you are asked to read a data which include 48505 articles (Documents). Then find the most similar documents using Locality Sensitive Hashing. Follow the lecture covering this topic step by step.

## 1. Data is available in Json format and you need to read it. 'https://www.ux.uis.no/~vsetty/data/assignment2_aricles.json' (5 points)


In [ ]:
data = requests.get('https://www.ux.uis.no/~vsetty/data/assignment2_aricles.json').json()[0:100]

df = pb.DataFrame.from_dict(data)

print(df)

## 2. Shingle the documents (10 points)
### Tips:
* Use string package to cleanup the articles e.g, str.maketrans('', '', string.
punctuation)
* It is better to convert text to lower case that way you get fewer n-grams
* apply ngrams(x.split(), n) using ngrams from nltk on the content + title for computing n-grams, for this data n = 2 is suffcient
  * You can use n-gram at word level for this task
  * try with different n-gram values 
  * You can use ngrams from nltk for this



In [ ]:
def cleanup(s):
    t = str.maketrans('', '', string.punctuation+"“”‘’")
    return s.translate(t).lower()

def ngram_text(s):
    n = ngrams(nltk.word_tokenize(s), 2)
    c = collections.Counter(n)
    m=c.most_common(10_000)
    return [ w[0] for w in m]



df['Title'] =  df['Title'].apply(cleanup)
df['Content'] =  df['Content'].apply(cleanup)


text_df = pb.DataFrame({})
text_df["Article"] = df['Title']+" "+df['Content']

shingles_all_articles = pb.DataFrame({})
shingles_all_articles["Shingles"] = [text_df['Article'].str.cat(sep=' ')]

shingles_all_articles = shingles_all_articles["Shingles"].apply(ngram_text)[0]

shingles_each_article = pb.DataFrame({})
shingles_each_article["Shingles"] = text_df['Article'].apply(ngram_text)


#print(shingles_all_articles)
#print(shingles_each_article)




## 3. Convert n-grams into binary vector representation for each document. You can do some optimzations if the matrix is too big. (10 points)
* For example,

  * Select top 10000 most frequent n-grams.
  * You may also try smaller values of n (like 2 or 3) which result in fewer n-grams.
  * Finally, you can also try sparse matrix representation. Like csr_matrix from scipy.sparse. It works even with full vocabulary.
    * Given a list of n-grams for each document, see how to builid a sparse matrix here https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html



In [ ]:

def to_binary_vec(article_shingles,all_shingles):
    column, row = [], []

    for c, article_ngrams in article_shingles['Shingles'].items():
        article_ngrams = set(article_ngrams)
        for r in range(len(all_shingles)):
            if all_shingles[r] in article_ngrams:
                column.append(c)
                row.append(r)


    return csr_matrix((len(row)*[1], (row, column)),shape=(len(all_shingles),len(article_shingles["Shingles"])))


matrix = to_binary_vec(shingles_each_article,shingles_all_articles)

matrix = pb.DataFrame(matrix.todense())
matrix



## 4. We need hash function that maps integers 0, 1, . . . , k − 1 to bucket numbers 0 through k − 1. It might be impossible to avoid collisions but as long as the collions are too many it won't matter much. (10 points)
* The simplest would be using the builtin hash() function, it can be for example, hash(rownumber) % Numberofbuckets
* You can generate several of these hash functions by xoring a random integer (hash(rownumber)^randint) % Numberofbuckets
* It can also be a as simple as (rownumber * randint) % Numberofbuckets



In [ ]:
random.seed(1)



def make_hash_func():
    a = list(range(1,10_001))
    random.shuffle(a)
    return a

hash_funcs = []
for _ in range(20):
    hash_funcs.append(make_hash_func())
    
pb.DataFrame(hash_funcs)





In [ ]:
def hash_matrix(row):
    signature = []
    for func in hash_funcs:
        for i in range(1,10_001):
            if row[func.index(i)] == 1:               
                signature.append(i)
                break
    return signature

signed_matrixes = matrix.apply(hash_matrix,axis=0)
signed_matrixes



## 5. Compute minhash following the faster algorithm from the lecture (10 points)


In [53]:
num_bands = 10
num_buckets = 200
def split_vec_and_hash(row):
    row = list(row)
    size = len(row)//num_bands
    return [hash(tuple(row[i*size:(i+1)*size]))%num_buckets for i in range(num_bands)]


split_signatures = signed_matrixes.apply(split_vec_and_hash,axis=0)

print(split_signatures)

buckets = {}

def find_collision(column):
    for i in range(len(column)):
        v = column[i]
        if v not in buckets:
            buckets[v] = set()
            # for each of the elements already in the bucket, add yourself to their nearest neighbors list
        buckets[v].add(i)


split_signatures.apply(find_collision,axis=1)

for i in range(num_buckets):
    if i in buckets:
        print(f"{i} -> {buckets[i]}")




# find nearest neighbours, meaning for each index find if any other index is in the same bucket

# calculate jaccard similarity on binary matrix for all nearest neighbours







    0    1    2    3    4    5    6    7    8    9   ...   90   91   92   93  \
0  174  152   45   52  183  166  119   14   96   88  ...   24  167   30   26   
1   19  191   45   82  167   66  150   79  172   14  ...  154  170   63  182   
2   52  147    8    4   61   99  141  148   58  173  ...  161  179  139  132   
3  139  144  184  183  153   72   55  128   34  187  ...  170  159   91  191   
4   89   20  134  129  129   32   67  161   98    5  ...  150  127  172  177   
5  112  112   61    7  161  100  158   28   60    4  ...   13  107   37   48   
6  161    8  102   56  159   23  154  111   85   52  ...  139  154   88  190   
7   96   41   13   96  170  193   29   50   69   55  ...  185   24  184  103   
8   62  141  167  119  198   50   95  184  164  131  ...   48   80   38   36   
9  157  123   63  145  111  123   36  128  137   59  ...   95   37  123   69   

    94   95   96   97   98   99  
0  188  158  172   67   34    3  
1   24   90   82  165  132  114  
2   66   57    8 

## 6. Hash signature bands into buckets. Find a way to combine all the  signature values in a band and hash them into a number of buckets ususally very high. (10 points)
* Easiest way is to add all the signature values in the bucket and use a similar hash function like before
* You should use the same hash function for all bands. And all documents ending up in same bucket for at least one band are considered as candidate pairs.



## 7. Tune parameters to make sure the threshold is appropriate. (10 points)
* plot the probability of two similar items falling in same bucket for different threshold values



## 8. Choose the best parameters and get nearest neighbors of each articles (20 points)
* Jaccard Similarity
* convert hash table into dictionary of article ids and its other articles that hashed in at least 1 same bucket



## 9. Write the nearest neibhors of each document to submissions.csv (comma separated, first column is the current document followed by a list of nearest neighbors) file and get the score (10 points)


## 10. Write a report + notebook + submission file in a zip file (5 points)


In [ ]:
import pandas as pd
import numpy as np
import json
import nltk
from nltk import ngrams


In [ ]:
def getFrequentNgrams(articles):
  # Your code Here
    # Select most frequent n-grams
   

In [ ]:
def getBinaryMatrix(docs):
  # Your code Here
       # return binary_matrix

In [ ]:
def getHashFunctionValues(numrows, numhashfunctions):
    # Your code Here
    #return a matrix with hash values


In [ ]:
def getMinHashSignatureMatrix(binary_matrix, hash_val_matrix):
    #return minhash signature matrix
    

In [ ]:
def getLSH(signature_matrix, num_bands, num_buckets):
    #return lsh buckets or hash table
    

In [ ]:
def plotProbability(s, b, r):
   

In [ ]:
def getJaccardSimilarityScore(C1, C2):
    

In [ ]:
# convert hash table into dictionary of article ids and its other articles that hashed in at least 1 same bucket
nearest_neighbors = {}


In [ ]:
# Remove the neighbors in same buckets but have similarity score < threshold s
n_copy = copy.deepcopy(nearest_neighbors)
submission_id = []
submission_nid = []
for article_id, neighbor_ids in n_copy.items():
    for nid in neighbor_ids:
        score = 
        if score < s:
           
        else:
            # add to submission result
            

In [ ]:
data = pd.DataFrame()
data['article_id'] = submission_id
data['neighbor_id'] = submission_nid
data.sort_values(by=['article_id', 'neighbor_id'], inplace=True)

In [ ]:
data.head(100)